In [3]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd

In [4]:
load_dotenv()
api_key = os.getenv("BHL_API_KEY")

In [14]:
volume_ids = ['135496','135963','135969','135967','135968']

In [15]:
bhl_results = []
for volume_id in volume_ids:
    r = requests.get('https://www.biodiversitylibrary.org/api3',
                     params={'apikey':api_key,
                             'op':'GetItemMetadata',
                             'id':volume_id,
                             'pages':'t',
                             'format':'json'})
    bhl_json = r.json()
    volume = bhl_json['Result'][0]['Volume']
    for page in bhl_json['Result'][0]['Pages']:
        page_id = page['PageID']
        page_type = '|'.join([pagetype['PageTypeName'] for pagetype in page['PageTypes']])
        if len(page['PageNumbers']):
            page_number = page['PageNumbers'][0]['Prefix'] + ' ' + page['PageNumbers'][0]['Number']
        else:
            page_number = ''
        bhl_results.append({'volume':volume,
                            'page_id':page_id,
                            'page_type':page_type,
                            'page_number':page_number})
        
len(bhl_results)

1338

In [16]:
bhl_df = pd.DataFrame(bhl_results)
bhl_df.head()

,volume,page_id,page_type,page_number
0,v.1 (1925),42602750,Cover,
1,v.1 (1925),42602749,Blank,
2,v.1 (1925),42602748,Text,
3,v.1 (1925),42602747,Blank,
4,v.1 (1925),42602746,Blank,


In [17]:
bhl_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   volume       1338 non-null   object
 1   page_id      1338 non-null   int64 
 2   page_type    1338 non-null   object
 3   page_number  1338 non-null   object
dtypes: int64(1), object(3)
memory usage: 41.9+ KB


In [18]:
bhl_df['page_type'].value_counts()

page_type
Blank                477
Text                 415
Illustration         401
Index                 15
Table of Contents     15
Cover                 10
Title Page             5
Name: count, dtype: int64

In [19]:
bhl_df.to_csv('walcott_bhl_api_results.tsv', sep='\t', index=False)